# ETL Weather DFs

In [19]:
import pandas as pd
import os
import numpy as np

### Aggregate station data by distance

In [3]:
pd.options.display.max_rows = 10
cities = ["ATL_stations","CH_stations","LA_stations","NYC_stations","SD_stations","SF_stations"]
#Aggregate Station Location tuples 
stations_data = pd.DataFrame()
for city in cities:
    path = 'station_locations/%s.txt' % city
    if os.path.exists(path):
        frame = pd.read_csv(path,delim_whitespace=True,header=None,error_bad_lines=False)
        frame['city'] = city
        stations_data = stations_data.append(frame,ignore_index=True)
stations_data = stations_data.rename(columns={0:"distance",1:"station_name"})

In [4]:
ATLshortlist = stations_data[(stations_data.city == "ATL_stations") & (stations_data.distance <= 100)][['station_name']]
CHshortlist = stations_data[(stations_data.city == "CH_stations") & (stations_data.distance <= 100)][['station_name']]
LAshortlist = stations_data[(stations_data.city == "LA_stations") & (stations_data.distance <= 100)][['station_name']]
NYCshortlist = stations_data[(stations_data.city == "NYC_stations") & (stations_data.distance <= 100)][['station_name']]
SDshortlist = stations_data[(stations_data.city == "SD_stations") & (stations_data.distance <= 100)][['station_name']]
SFshortlist = stations_data[(stations_data.city == "SF_stations") & (stations_data.distance <= 100)][['station_name']]

### Transform and Load Data 
###### Merge datasets from each station with PARAM_STATION-NAME as default column header
##### Process results in 10 stations per city
##### UPDATE: Frame.query removes scrappy data (missing data will still exist for some!) 

In [21]:
######ATL Weather######
ATL_stations = [];
ATL_weather = pd.DataFrame()
for station in ATLshortlist['station_name']:
    path = 'ATL/%s.csv' % station
    if os.path.exists(path):
        frame = pd.read_csv(path,delim_whitespace=True)
        frame = frame.query('TMIN != TMAX')
        frame.columns = ['DATE', 'TMAX_'+station,'TMIN_'+station,'SNOW_'+station,'SNWD_'+station,'PRCP_'+station]
        if frame.shape[0] >= 12650:
            ATL_stations.append(station);
            if ATL_weather.empty:
                ATL_weather = ATL_weather.append(frame,ignore_index=True)
            else:
                ATL_weather = ATL_weather.merge(frame, on='DATE', how='outer', suffixes=('',''))
ATL_weather = ATL_weather.groupby('DATE').mean()

In [49]:
######CH Weather######
CH_stations = [];
CH_weather = pd.DataFrame()
for station in CHshortlist['station_name']:
    path = 'CH/%s.csv' % station
    if os.path.exists(path):
        frame = pd.read_csv(path,delim_whitespace=True)
        frame = frame.query('TMIN != TMAX')
        frame.columns = ['DATE', 'TMAX_'+station,'TMIN_'+station,'SNOW_'+station,'SNWD_'+station,'PRCP_'+station]
        if frame.shape[0] >= 12764:
            CH_stations.append(station)
            if CH_weather.empty:
                CH_weather = CH_weather.append(frame,ignore_index=True)
            else:
                CH_weather = CH_weather.merge(frame, on='DATE', how='outer', suffixes=('',''))
CH_weather = CH_weather.groupby('DATE').mean()

In [42]:
######NYC Weather######
NYC_stations = [];
NYC_weather = pd.DataFrame()
for station in NYCshortlist['station_name']:
    path = 'NYC/%s.csv' % station
    if os.path.exists(path):
        frame = pd.read_csv(path,delim_whitespace=True)
        frame = frame.query('TMIN != TMAX')
        frame.columns = ['DATE', 'TMAX_'+station,'TMIN_'+station,'SNOW_'+station,'SNWD_'+station,'PRCP_'+station]
        if frame.shape[0] >= 12700:
            NYC_stations.append(station)
            if NYC_weather.empty:
                NYC_weather = NYC_weather.append(frame,ignore_index=True)
            else:
                NYC_weather = NYC_weather.merge(frame, on='DATE', how='outer', suffixes=('',''))
NYC_weather = NYC_weather.groupby('DATE').mean()

In [23]:
######LA Weather######
LA_stations = [];
LA_weather = pd.DataFrame()
for station in LAshortlist['station_name']:
    path = 'LA/%s.csv' % station
    if os.path.exists(path):
        frame = pd.read_csv(path,delim_whitespace=True)
        frame = frame.query('TMIN != TMAX')
        frame.columns = ['DATE', 'TMAX_'+station,'TMIN_'+station,'SNOW_'+station,'SNWD_'+station,'PRCP_'+station]
        if frame.shape[0] >= 12600: 
            LA_stations.append(station)
            if LA_weather.empty:
                LA_weather = LA_weather.append(frame,ignore_index=True)
            else:
                LA_weather = LA_weather.merge(frame, on='DATE', how='outer', suffixes=('',''))
LA_weather = LA_weather.groupby('DATE').mean()

In [24]:
######SF Weather######
SF_stations = [];
SF_weather = pd.DataFrame()
for station in SFshortlist['station_name']:
    path = 'SF/%s.csv' % station
    if os.path.exists(path):
        frame = pd.read_csv(path,delim_whitespace=True)
        frame = frame.query('TMIN != TMAX')
        frame.columns = ['DATE', 'TMAX_'+station,'TMIN_'+station,'SNOW_'+station,'SNWD_'+station,'PRCP_'+station]
        if frame.shape[0] >= 12784: 
            SF_stations.append(station)
            if SF_weather.empty:
                SF_weather = SF_weather.append(frame,ignore_index=True)
            else:
                SF_weather = SF_weather.merge(frame, on='DATE', how='outer', suffixes=('',''))
SF_weather = SF_weather.groupby('DATE').mean()

In [52]:
######SD Weather######
SD_stations = [];
SD_weather = pd.DataFrame()
for station in SDshortlist['station_name']:
    path = 'SD/%s.csv' % station
    if os.path.exists(path):
        frame = pd.read_csv(path,delim_whitespace=True)
        frame = frame.query('TMIN != TMAX')
        frame.columns = ['DATE', 'TMAX_'+station,'TMIN_'+station,'SNOW_'+station,'SNWD_'+station,'PRCP_'+station]
        if frame.shape[0] >= 12600: 
            SD_stations.append(station)
            if SD_weather.empty:
                SD_weather = SD_weather.append(frame,ignore_index=True)
            else:
                SD_weather = SD_weather.merge(frame, on='DATE', how='outer', suffixes=('',''))
SD_weather = SD_weather.groupby('DATE').mean()

### Missing Data Logger
##### True indicates a missing data row 

In [51]:
print("New York City stations missing data")
for station in NYC_stations:
    print("Station %s" % station)
    print(pd.isnull(NYC_weather)['TMIN_'+station].value_counts())
    print()

print("Atlanta stations missing data")
for station in ATL_stations:
    print("Station %s" % station)
    print(pd.isnull(ATL_weather)['TMIN_'+station].value_counts())
    print()
    
print("San Francisco stations missing data")
for station in SF_stations:
    print("Station %s" % station)
    print(pd.isnull(SF_weather)['TMIN_'+station].value_counts())
    print()

print("San Diego stations missing data")
for station in SD_stations:
    print("Station %s" % station)
    print(pd.isnull(SD_weather)['TMIN_'+station].value_counts())
    print()

print("Los Angeles stations missing data")
for station in LA_stations:
    print("Station %s" % station)
    print(pd.isnull(LA_weather)['TMIN_'+station].value_counts())
    print()

    print("Chicago stations missing data")
for station in CH_stations:
    print("Station %s" % station)
    print(pd.isnull(CH_weather)['TMIN_'+station].value_counts())
    print()    

New York City stations missing data
Station USC00300889
False    12747
True        37
dtype: int64

Station USC00305426
False    12782
True         2
dtype: int64

Station USW00014732
False    12784
dtype: int64

Station USW00094728
False    12784
dtype: int64

Station USW00094745
False    12691
True        93
dtype: int64

Station USW00094789
False    12784
dtype: int64

Station USC00283951
False    12762
True        22
dtype: int64

Station USC00286055
False    12781
True         3
dtype: int64

Station USW00014734
False    12784
dtype: int64

Station USW00093730
False    12784
dtype: int64

Atlanta stations missing data
Station USC00092283
False    12784
dtype: int64

Station USC00092485
False    12766
True        18
dtype: int64

Station USC00093060
False    12779
True         5
dtype: int64

Station USC00093621
False    12749
True        35
dtype: int64

Station USW00003813
False    12784
dtype: int64

Station USW00013873
False    12784
dtype: int64

Station USW00013874
False    1